In [12]:
import os
from pathlib import Path
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from modules.functions import bound_logistic
from modules.plotstyle import PlotStyle
import matplotlib.colors as mcolors
import plottools.colors as clrs
import cmocean.cm as cmo
import seaborn as sns
from scipy.stats import wilcoxon
from cvs_preprocessing import dataimport
import matplotlib.colors as mcolors
%matplotlib qt

# init standardized plotstyle
ps = PlotStyle()

In [13]:
# import the data
dataroot = Path("../data/cvs")
processed_dataroot = Path("../data_processed")
df = dataimport(dataroot, processed_dataroot)

# delete JONA!
df = df[df.subj != 'VP1']

# means and sem for trial duration
names = np.unique(df.subj).tolist()
n = df.shape[1] / len(names)
means = np.array(df.tpertrial.groupby(df.subj).mean())
sems = np.array(df.tpertrial.groupby(df.subj).std()) / np.sqrt(n)

# put to dataframe
df_all = pd.DataFrame(
    list(zip(names, means, sems)),
    columns=("names", "tpertrial_means", "tpertrial_sems"),
)

# means and sems for no of switches
means = np.array(df.switches.groupby(df.subj).mean())
sems = np.array(df.switches.groupby(df.subj).std()) / np.sqrt(n)
df_all["switches_means"] = means
df_all["switches_sems"] = sems


In [14]:
# fit power law function
def func_powerlaw(x, m, c, c0):
    return c0 + x**m * c

popt, pcov = curve_fit(func_powerlaw, df_all.switches_means, df_all.tpertrial_means, method="trf", maxfev=50000)




In [15]:
# plot data for all subjects
colors = mcolors.TABLEAU_COLORS

fig, ax = plt.subplots(1,2, figsize=(24*ps.cm, 12*ps.cm), constrained_layout=True)
for i in range(len(ax)):

    for name, color in zip(df_all.names, colors):
        minidf = df_all[df_all.names == name]
        ax[i].errorbar(
            minidf.switches_means,
            minidf.tpertrial_means,
            xerr=minidf.switches_sems,
            yerr=minidf.tpertrial_sems,
            fmt="bo",
            label=name,
            color=color,
        )

x = np.linspace(df_all.switches_means.min()-0.1, df_all.switches_means.max()+2, 1000)

ax[1].plot(x, func_powerlaw(x, *popt))
ax[0].plot(x, func_powerlaw(x, *popt))
ax[1].set_ylim(0, 6)

fig.supxlabel('Number of switches')
fig.supxlabel(' of switches')
plt.show()

/tmp/ipykernel_3847/213345687.py:9: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "bo" (-> color='b'). The keyword argument will take precedence.
  ax[i].errorbar(


In [16]:
# seperate dataset according to 